## Format for accessing options copied from here:

https://medium.com/@txlian13/webscrapping-options-data-with-python-and-yfinance-e4deb0124613

In [31]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.io import output_notebook
import yfinance as yf
import datetime

In [8]:
def options_chain(symbol):

    tk = yf.Ticker(symbol)
    # Expiration dates
    exps = tk.options

    # Get options for each expiration
    options = pd.DataFrame()
    for e in exps:
        opt = tk.option_chain(e)
        opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt["expirationDate"] = e
        options = options.append(opt, ignore_index=True)

    # Bizarre error in yfinance that gives the wrong expiration date
    # Add 1 day to get the correct expiration date
    options["expirationDate"] = pd.to_datetime(options["expirationDate"]) + datetime.timedelta(days = 1)
    options["dte"] = (options["expirationDate"] - datetime.datetime.today()).dt.days / 365
    
    # Boolean column if the option is a CALL
    options["CALL"] = options["contractSymbol"].str[4:].apply(
        lambda x: "C" in x)
    
    options[["bid", "ask", "strike"]] = options[["bid", "ask", "strike"]].apply(pd.to_numeric)
    options["mark"] = (options["bid"] + options["ask"]) / 2 # Calculate the midpoint of the bid-ask
    
    # Drop unnecessary and meaningless columns
    options = options.drop(columns = ["contractSize", "currency", "change", "percentChange", "lastTradeDate", "lastPrice"])

    return options

In [83]:
df = options_chain("AMD")

/var/folders/3f/m_d1fd_d0z13629b400kbmvr0000gn/T/ipykernel_90190/2545506964.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  opt = pd.DataFrame().append(opt.calls).append(opt.puts)
/var/folders/3f/m_d1fd_d0z13629b400kbmvr0000gn/T/ipykernel_90190/2545506964.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  options = options.append(opt, ignore_index=True)
/var/folders/3f/m_d1fd_d0z13629b400kbmvr0000gn/T/ipykernel_90190/2545506964.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  opt = pd.DataFrame().append(opt.calls).append(opt.puts)
/var/folders/3f/m_d1fd_d0z13629b400kbmvr0000gn/T/ipykernel_90190/2545506964.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

/var/folders/3f/m_d1fd_d0z13629b400kbmvr0000gn/T/ipykernel_90190/2545506964.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  opt = pd.DataFrame().append(opt.calls).append(opt.puts)
/var/folders/3f/m_d1fd_d0z13629b400kbmvr0000gn/T/ipykernel_90190/2545506964.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  options = options.append(opt, ignore_index=True)


In [84]:
df

,contractSymbol,strike,bid,ask,volume,openInterest,impliedVolatility,inTheMoney,expirationDate,dte,CALL,mark
0,AMD231117C00040000,40.0,0.0,0.0,2.0,122.0,0.000010,True,2023-11-18,0.002740,True,0.0
1,AMD231117C00045000,45.0,0.0,0.0,3.0,70.0,0.000010,True,2023-11-18,0.002740,True,0.0
2,AMD231117C00050000,50.0,0.0,0.0,10.0,89.0,0.000010,True,2023-11-18,0.002740,True,0.0
3,AMD231117C00055000,55.0,0.0,0.0,5.0,20.0,0.000010,True,2023-11-18,0.002740,True,0.0
4,AMD231117C00060000,60.0,0.0,0.0,2.0,176.0,0.000010,True,2023-11-18,0.002740,True,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1458,AMD260116P00180000,180.0,69.1,73.1,4.0,4.0,0.391791,True,2026-01-17,2.169863,False,71.1
1459,AMD260116P00185000,185.0,0.0,0.0,5.0,7.0,0.000010,True,2026-01-17,2.169863,False,0.0
1460,AMD260116P00190000,190.0,0.0,0.0,NaN,2.0,0.000010,True,2026-01-17,2.169863,False,0.0
1461,AMD260116P00200000,200.0,0.0,0.0,34.0,35.0,0.000010,True,2026-01-17,2.169863,False,0.0
